# 城市街道网络定位

使用OSMNX在城市路网确定位置

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd

ox.config(log_console=True, use_cache=True)
weight_by_length = False

ox.__version__

'0.11dev'

In [2]:
places = {'Dongcheng District'        : 'Dongcheng District,Beijing, CN',
          'Xicheng District'          : 'Xicheng District,Beijing, CN',
          'Haidian District'          : 'Haidian District,Beijing, CN',
          'Chaoyang District'         : 'Chaoyang District,Beijing, CN',
          'Daxing District'           : 'Daxing District,Beijing, CN',
          'Shijingshan District'      : 'Shijingshan District,Beijing, CN',
          'Tongzhou District'         : 'Tongzhou District,Beijing, CN',
          'Fangshan District'         : 'Fangshan District,Beijing, CN',
          'Changping District'        :'Changping District,Beijing, CN'
          }
which_results =  {'Dongcheng District'        : 1,
                  'Xicheng District'          : 1,
                  'Haidian District'          : 1,
                  'Chaoyang District'         : 1,
                  'Daxing District'           : 1,
                  'Shijingshan District'      : 1,
                  'Tongzhou District'         : 1,
                  'Fangshan District'         : 1,
                  'Changping District'        : 1
          }

In [3]:
# verify OSMnx geocodes each query to what you expect
gdf = ox.gdf_from_places(places.values(),which_results=which_results.values())
gdf

,bbox_east,bbox_north,bbox_south,bbox_west,geometry,place_name
0,116.444677,39.972647,39.856541,116.372275,"POLYGON ((116.37227 39.86584, 116.37248 39.865...","Dongcheng District, Beijing, 100010, China"
1,116.393426,39.971617,39.867536,116.314586,"POLYGON ((116.31459 39.87468, 116.31505 39.874...","Xicheng District, Beijing, 100032, China"
2,116.388826,40.159812,39.885308,116.042548,"POLYGON ((116.04255 40.08391, 116.04270 40.083...","Haidian District, China"
3,116.639273,40.109421,39.808340,116.344717,"MULTIPOLYGON (((116.34472 40.02534, 116.36883 ...","Chaoyang District, Beijing, China"
4,116.721057,39.830816,39.441611,116.209745,"POLYGON ((116.20975 39.64346, 116.21050 39.639...","Daxing District, Beijing, China"
5,116.253448,39.992302,39.872523,116.105796,"POLYGON ((116.10580 39.94129, 116.10937 39.937...","Shijingshan District, China"
6,116.948027,40.030624,39.592099,116.521020,"POLYGON ((116.52102 39.71592, 116.52638 39.714...","Tongzhou District, Beijing, 101100, China"
7,116.247257,39.923906,39.505835,115.418725,"POLYGON ((115.41872 39.77361, 115.42098 39.768...","Fangshan District, Beijing, China"
8,116.499969,40.391562,40.035298,115.838245,"POLYGON ((115.83825 40.16663, 115.83883 40.163...","Changping District, China"


##  获取街道网络及其边缘方位

In [5]:
def reverse_bearing(x):
    return x + 180 if x < 180 else x - 180

In [6]:
bearings = {}
for place in sorted(places.keys()):
    
    # get the graph
    query = places[place]
    which_result = which_results[place]
    G = ox.graph_from_place(query,which_result=which_result, network_type='drive',buffer_dist=250)
    
    # calculate edge bearings
    Gu = ox.add_edge_bearings(ox.get_undirected(G))
    
    if weight_by_length:
        # weight bearings by length (meters)
        city_bearings = []
        for u, v, k, d in Gu.edges(keys=True, data=True):
            city_bearings.extend([d['bearing']] * int(d['length']))
        b = pd.Series(city_bearings)
        bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
    else:
        # don't weight bearings, just take one value per street segment
        b = pd.Series([d['bearing'] for u, v, k, d in Gu.edges(keys=True, data=True)])
        bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')

## 可视化

In [7]:
def count_and_merge(n, bearings):
    # make twice as many bins as desired, then merge them in pairs
    # prevents bin-edge effects around common values like 0° and 90°
    n = n * 2
    bins = np.arange(n + 1) * 360 / n
    count, _ = np.histogram(bearings, bins=bins)
    
    # move the last bin to the front, so eg 0.01° and 359.99° will be binned together
    count = np.roll(count, 1)
    return count[::2] + count[1::2]

In [8]:
# function to draw a polar histogram for a set of edge bearings
def polar_plot(ax, bearings, n=36, title=''):

    bins = np.arange(n + 1) * 360 / n
    count = count_and_merge(n, bearings)
    _, division = np.histogram(bearings, bins=bins)
    frequency = count / count.sum()
    division = division[0:-1]
    width =  2 * np.pi / n

    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')

    x = division * np.pi / 180
    bars = ax.bar(x, height=frequency, width=width, align='center', bottom=0, zorder=2,
                  color='#003366', edgecolor='k', linewidth=0.5, alpha=0.7)
    
    ax.set_ylim(top=frequency.max())
    
    title_font = {'family':'Century Gothic', 'size':24, 'weight':'bold'}
    xtick_font = {'family':'Century Gothic', 'size':10, 'weight':'bold', 'alpha':1.0, 'zorder':3}
    ytick_font = {'family':'Century Gothic', 'size': 9, 'weight':'bold', 'alpha':0.2, 'zorder':3}
    
    ax.set_title(title.upper(), y=1.05, fontdict=title_font)
    
    ax.set_yticks(np.linspace(0, max(ax.get_ylim()), 5))
    yticklabels = ['{:.2f}'.format(y) for y in ax.get_yticks()]
    yticklabels[0] = ''
    ax.set_yticklabels(labels=yticklabels, fontdict=ytick_font)
    
    xticklabels = ['N', '', 'E', '', 'S', '', 'W', '']
    ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
    ax.tick_params(axis='x', which='major', pad=-2)

In [9]:
# create figure and axes
n = len(places)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={'projection':'polar'})

# plot each city's polar histogram
for ax, place in zip(axes.flat, sorted(places.keys())):
    polar_plot(ax, bearings[place].dropna(), title=place)

# add super title and save full image
suptitle_font = {'family':'Century Gothic', 'fontsize':60, 'fontweight':'normal', 'y':1.07}
fig.suptitle('City Street Network Orientation', **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
fig.savefig('images/street-orientations.png', dpi=120, bbox_inches='tight')
plt.close()